In [1]:
import pandas as pd
import random
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

def get_youtube_comments(video_url, max_comments=100, max_scrolls=10):
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--lang=ko-KR')
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(20)

    try:
        driver.get(video_url)
        time.sleep(3)

        scrolls = 0
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        for _ in range(max_scrolls):  # ✅ 최대 10번까지만 시도
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height or len(driver.find_elements(By.ID, 'content-text')) >= max_comments:
                break
            last_height = new_height
            scrolls += 1

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        comment_elems = soup.select('#content-text')
        comments = [c.text.strip() for c in comment_elems[:max_comments]]
    
    except Exception as e:
        print(f"❌ 댓글 로딩 중 예외 발생: {video_url}\n→ {e}")
        comments = []

    finally:
        driver.quit()

    return comments

def extract_video_id(url):
    parsed = urlparse(url)
    return parse_qs(parsed.query).get('v', [url.split("v=")[-1]])[0]

if __name__ == '__main__':
    input_file = 'youtube_metadata_김문수_후보_등록.csv'
    df = pd.read_csv(input_file)

    all_data = []

    for i, row in df.iterrows():
        video_url = row['url']
        video_id = extract_video_id(video_url)
        print(f"\n🎬 {i+1}/{len(df)} | {row['title']} | 채널: {row['channel']} | ID: {video_id}")

        try:
            comments = get_youtube_comments(video_url, max_comments=100)
            print(f"🗨️ {len(comments)}개 댓글 수집 완료")

            # ✅ 댓글 미리보기 3개
            if comments:
                print("📌 댓글 미리보기:")
                for preview in comments[:3]:
                    print(f"   - {preview}")
            else:
                print("⚠️ 댓글 없음 또는 비활성화 영상")

            for c in comments:
                all_data.append({
                    'videoId': video_id,
                    'title': row['title'],
                    'channel': row['channel'],
                    'comment': c
                })

        except Exception as e:
            print(f"❌ 오류 발생: {video_id} → {e}")

        # ✅ 영상 간 대기시간 랜덤화
        time.sleep(4 + random.uniform(0, 2))

    df_comments = pd.DataFrame(all_data)

    # ✅ 최종 미리보기
    print("\n📝 최종 댓글 통합 미리보기:")
    print(df_comments.head(3))

    df_comments.to_csv('youtube_comments_김문수_후보_등록.csv', index=False, encoding='utf-8-sig')
    print(f"\n✅ 총 댓글 수집 완료: {len(df_comments)}건 → youtube_comments_김문수_후보_등록.csv")



🎬 1/50 | 김문수 후보등록 백브리핑 - 권영세가 사퇴한다 해놓고 사임을 안해~ 너 나가~~ | 채널: 유재일 | ID: 2NcIw1Ow3ss
🗨️ 8개 댓글 수집 완료
📌 댓글 미리보기:
   - 한동훈대표님께 감사해야합니다
   - 새로운 모습을 보았습니다 기적이 일어날거라 봅니다
   - 아이구 답답

🎬 2/50 | 대선 판도 뒤흔드는 김문수, 등록과 갈등의 종지부! 김문수 대선 후보 등록 소식! | 채널: 생생정보 | ID: dnEa11ahuxo
🗨️ 0개 댓글 수집 완료
⚠️ 댓글 없음 또는 비활성화 영상

🎬 3/50 | 김문수, 대선 후보 등록! “기적이 일어났다”… 한덕수 전 총리와 만날 예정 | 채널: 팩트스팟FactSpot | ID: Cx8MeN9iG9E
🗨️ 2개 댓글 수집 완료
📌 댓글 미리보기:
   - 하룻밤이 불안한 당~
하룻밤이 불안한 나이~~
   - 승리절대못한다

🎬 4/50 | #리셋코리아 #대선후보등록#김문수 대통령 후보 | 채널: MARS | ID: iDBEVaw7mBI
🗨️ 1개 댓글 수집 완료
📌 댓글 미리보기:
   - 승리합시다.자유민주주의 체재 수호 못하면 베네주엘라 직행입니다.

🎬 5/50 | #김문수, 선관위 후보등록…"반드시 당선돼 위대한 나라 만들 것""당원 투표, 놀라운 기적 일어나…한덕수, 잘 모시겠다"지도부 책임론에 "선거 며칠 안남아 화합해야, 빅텐트 중요 | 채널: 바람소리 : 진거무선 | ID: n7Ykax92B0I
🗨️ 2개 댓글 수집 완료
📌 댓글 미리보기:
   - 힘내세요
   - 뭐 하나 자연스럽지 않은 등록ᆢ
오물통속 등록ᆢ

🎬 6/50 | 국민의힘, 결국 김문수 후보로 대선 후보 등록하기로..권영세 "모든 책임지고 물러나겠다" [이슈PLAY] / JTBC News | 채널: JTBC News | ID: xzv1e7Yr5RY
🗨️ 10개 댓글 수집 완료
📌 댓글 미리보기:
   - 극짐 내란당이 보수로보임?  제발좀 보수와 골수의 차이를 생각좀해요. 저들은 골수입니다. 자기